In [1]:
from dataset_utils import nb_channels, encode_position, store_many_hdf5
from ScoreGetter import ScoreGetter
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("E:/IA/Deep_ViCTORIA/Datasets/fen_dataset.csv", nrows=200000)
df = df.sample(frac=1)

In [3]:
score_getter = ScoreGetter("stockfish.exe", "eval", "go depth 1")

nb_position = 100000
position = np.zeros((nb_position, 8, 8, nb_channels), dtype=np.float32)
scores = np.zeros(nb_position, dtype=np.float32)
count = 0
for i in range(nb_position):
  fen = df["board"][i]
  try:
    scores[i] = score_getter.getScore(fen)
    position[i] = encode_position(fen)
    count += 1
  except Exception as e:
    if str(e) == "[Errno 32] Broken pipe":
        score_getter.restart()
    continue
scores = scores[:count].reshape(-1, 1)
position = position[:count]
print(f"Number of positions: {position.shape[0]}")

Number of positions: 99296


In [4]:
directory = "E:/IA/Deep_ViCTORIA/Datasets/SE_ResNet/"
store_many_hdf5(position, scores, directory, tag="_test")